In [2]:
import pandas as pd
import tarfile
from tqdm import tqdm
import json
import numpy as np
from copy import deepcopy
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
from transformers import RobertaTokenizer, BertTokenizer, TFRobertaModel
import implicit
import matplotlib.pyplot as plt
import scipy.sparse as sparse
import tensorflow as tf
import pydot
import graphviz


In [3]:
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
numpyArray = np.load('../Jiro/features.npy')
np.load = np_load_old
ratings_ = df = pd.DataFrame(numpyArray, columns = ['user_id','business_id','stars','text'])

In [4]:
ratingsnew = ratings_.head(10000)
ratingsnew["stars"].astype(float).round()

for count in range(len(ratingsnew.text)):
    ratingsnew.at[count, 'text'] = " ".join(ratingsnew.at[count, 'text'])

ratingsnew['text'] = ratingsnew['text'].astype(str)
print(ratings_.head(1000))


                    user_id             business_id stars  \
0    mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   3.0   
1    OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   5.0   
2    8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   3.0   
3    _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   5.0   
4    bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   4.0   
..                      ...                     ...   ...   
995  syyKcKPFILDysHWmtka-aA  1_hDCN3iioFR3XnUr32ZtA   5.0   
996  _lgLNzpzf3qmbwySBakxEw  5RzJ2bjU8bLSaN5SuiUpYA   4.0   
997  Sh_vUlHHY2Kuj14eF8NYZQ  s1PNBO9o5jIgNd5YWUDLXQ   5.0   
998  YwMD-AVT67fmYRGxnlRSPA  alUk6OwNhofyc90NDMDY-Q   5.0   
999  WKe2b_EeLBnZ3lZV5WKYGQ  -Or44IdY51Ukd618kikmtA   4.0   

                                                  text  
0    [CLS] If you decide to eat here , just be awar...  
1    [CLS] I ' ve taken a lot of spin classes over ...  
2    [CLS] Family diner . Had the b ##uff ##et . E ...  
3    [CLS] Wow ! Yu ##mmy , different ,

C:\Users\apran\AppData\Local\Temp\ipykernel_20344\3648199601.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratingsnew['text'] = ratingsnew['text'].astype(str)


In [5]:
df_features = ratingsnew
df_features["user_id"]  = df_features['user_id'].astype("category").cat.codes
df_features["business_id"]  = df_features['business_id'].astype("category").cat.codes

C:\Users\apran\AppData\Local\Temp\ipykernel_20344\1655519412.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features["user_id"]  = df_features['user_id'].astype("category").cat.codes
C:\Users\apran\AppData\Local\Temp\ipykernel_20344\1655519412.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features["business_id"]  = df_features['business_id'].astype("category").cat.codes


In [6]:

sparse_item_user = sparse.csr_matrix((df_features['stars'].astype('float'), (df_features['user_id'], df_features['business_id'])))
sparse_user_item = sparse.csr_matrix((df_features['stars'].astype('float'),  (df_features['business_id'],df_features['user_id'])))

In [7]:
modelRec = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)


alpha_val = 40
data_conf = (sparse_item_user * alpha_val).astype('double')


modelRec.fit(data_conf)

c:\Users\apran\anaconda3\envs\nlp\lib\site-packages\implicit\utils.py:33: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

In [8]:
df_features["labels"] = df_features.apply(lambda row: modelRec.user_factors[row['user_id']],axis=1)
df_features.head()

C:\Users\apran\AppData\Local\Temp\ipykernel_20344\3483575756.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features["labels"] = df_features.apply(lambda row: modelRec.user_factors[row['user_id']],axis=1)


,user_id,business_id,stars,text,labels
0,7445,2091,3.0,"[CLS] If you decide to eat here , just be awar...","[0.96263444, 1.5302861, -0.41343868, -0.850829..."
1,3849,525,5.0,[CLS] I ' ve taken a lot of spin classes over ...,"[-1.2514894, -0.6296294, 1.5534954, 2.5624545,..."
2,1395,2179,3.0,[CLS] Family diner . Had the b ##uff ##et . E ...,"[-0.049234014, 0.051309265, 0.07644286, -0.012..."
3,5467,2992,5.0,"[CLS] Wow ! Yu ##mmy , different , delicious ....","[1.2380933, 1.1503192, 1.4515795, 0.7673191, 2..."
4,5864,2555,4.0,[CLS] Cut ##e interior and owner ( ? ) gave us...,"[1.6977916, 0.90207666, -0.53705466, 0.3739318..."


In [9]:
train_df, test_df = train_test_split(df_features, test_size = 0.2)
y_train = tf.convert_to_tensor(
    train_df["labels"].tolist(), dtype=float
)

y_test = tf.convert_to_tensor(
    test_df["labels"].tolist(), dtype=float
)

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
roberta = TFRobertaModel.from_pretrained('roberta-base')

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [11]:
x_train = tokenizer(
    text=train_df.text.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=test_df.text.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [12]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [13]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsolutePercentageError
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [14]:
max_len = 70
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = roberta(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(20,activation = 'linear')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True


In [15]:
optimizer = Adam(
    learning_rate=5e-05, 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss = MeanSquaredError()
metric = MeanSquaredError()
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = [MeanSquaredError()])

In [16]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=1,
    batch_size=36
)

223/223 [==============================] - 1035s 5s/step - loss: 1.1311 - mean_squared_error: 1.1312 - val_loss: 1.1253 - val_mean_squared_error: 1.1270


In [17]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

63/63 [==============================] - 84s 1s/step


array([ 0.24781121,  0.11145668,  0.21534464,  0.33749267,  0.21300954,
        0.00316622,  0.22932123, -0.00125662,  0.00200173,  0.07583589,
        0.11323715,  0.0272106 , -0.0690098 ,  0.11480001,  0.05543702,
        0.06659418,  0.05792034,  0.21053341,  0.02901679,  0.06194955],
      dtype=float32)

In [ ]:
y_test[0]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([-0.86721027, -1.7936006 ,  0.84505516,  0.85846627,  0.93760675,
        0.425725  , -0.9982252 , -0.4950628 ,  1.1601727 ,  1.1762882 ,
       -1.5319097 ,  1.7960157 , -1.6020365 ,  1.019224  ,  1.0259027 ,
        0.5282015 ,  0.59260374,  1.5099036 , -1.4663506 ,  0.916149  ],
      dtype=float32)>

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file='model.png',
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False
)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [21]:
model.save("./roberta/")

INFO:tensorflow:Assets written to: ./roberta/assets


INFO:tensorflow:Assets written to: ./roberta/assets
